# 5.9. Wykorzystanie KNN do klasyfikacji

Znamy już podstawy algorytmu K-Nearest Neighbours. Wykorzystajmy tę metodę, żeby nauczyć kolejny model przewidujący default na karcie kredytowej. Wczytajmy zbiór przygotowany wcześniej, dla którego przeprowadziliśmy już wstępne przetworzenie oraz przefiltrowanie cech, jakie model regresji logistycznej uznał za istotne.

In [1]:
import pandas as pd

In [2]:
credit_cards_df = pd.read_parquet("../data/credit-cards-reduced.parquet")
credit_cards_df.sample(n=5).T

ID,22626,7045,20757,10624,16675
AGE,34.000000,42.00,46.000000,27.000000,48.000000
PAY_1,2.000000,-1.00,2.000000,0.000000,0.000000
PAY_2,2.000000,-1.00,2.000000,0.000000,0.000000
PAY_3,2.000000,-1.00,2.000000,0.000000,2.000000
PAY_4,2.000000,-1.00,3.000000,2.000000,2.000000
PAY_5,2.000000,-1.00,2.000000,0.000000,2.000000
PAY_6,2.000000,-1.00,3.000000,0.000000,2.000000
PAY_OVERDUE_COUNT,6.000000,0.00,6.000000,1.000000,4.000000
WEIGHTED_PAYMENT_HISTORY,4.900000,-2.45,5.316667,0.500000,1.900000
AVG_PAY_AMT,13600.166667,386.00,1579.833333,1666.666667,1416.666667


Nie wiemy jednak początkowo, jak wiele sąsiadujących obserwacji powinien rozważyć model. Przetestujmy kilka wartości oraz wykonajmy dodatkowo walidację krzyżową. Tym razem musimy podzielić nasz zbiór na treningowy, walidacyjny oraz testowy. Pierwsze dwa zbiory zostaną niejako samodzielnie utworzone podczas procedury CV.

In [3]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split

In [4]:
X_train, X_test = train_test_split(credit_cards_df,
                                   test_size=0.2, 
                                   random_state=2020)

In [5]:
grid_cv = GridSearchCV(KNeighborsClassifier(), 
                       param_grid={
                           "n_neighbors": range(1, 10)
                       }, cv=5, scoring="f1", 
                       verbose=1, n_jobs=6)

In [6]:
grid_cv.fit(X_train.drop(columns="DEFAULT"), 
            X_train["DEFAULT"])

Fitting 5 folds for each of 9 candidates, totalling 45 fits


[Parallel(n_jobs=6)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=6)]: Done  45 out of  45 | elapsed:    6.9s finished


GridSearchCV(cv=5, error_score=nan,
             estimator=KNeighborsClassifier(algorithm='auto', leaf_size=30,
                                            metric='minkowski',
                                            metric_params=None, n_jobs=None,
                                            n_neighbors=5, p=2,
                                            weights='uniform'),
             iid='deprecated', n_jobs=6,
             param_grid={'n_neighbors': range(1, 10)}, pre_dispatch='2*n_jobs',
             refit=True, return_train_score=False, scoring='f1', verbose=1)

In [7]:
grid_cv.best_score_

0.31372138700274566

In [8]:
knn = grid_cv.best_estimator_
knn

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=1, p=2,
                     weights='uniform')

In [9]:
from sklearn.metrics import f1_score

In [10]:
f1_score(X_test["DEFAULT"],
         knn.predict(X_test.drop(columns="DEFAULT")))

0.3337250293772033

Wiemy już jaka konfiguracja modelu zdaje się dawać najlepsze rezultaty. Ciągle jest ona drastycznie niższa niż dla regresji logistycznej.

**Popełniliśmy jednak pewien istotny błąd!**

## Skalowanie zmiennych, a KNN

Kilkukrotnie wspomnieliśmy już o potrzebie skalowania zmiennych. Zobaczmy, jak kształtują się wartości w naszym zbiorze.

In [11]:
credit_cards_df.describe().T

,count,mean,std,min,25%,50%,75%,max
AGE,30000.0,35.485500,9.217904,21.0,28.000000,34.000000,41.000000,79.000000
PAY_1,30000.0,-0.016700,1.123802,-2.0,-1.000000,0.000000,0.000000,8.000000
PAY_2,30000.0,-0.133767,1.197186,-2.0,-1.000000,0.000000,0.000000,8.000000
PAY_3,30000.0,-0.166200,1.196868,-2.0,-1.000000,0.000000,0.000000,8.000000
PAY_4,30000.0,-0.220667,1.169139,-2.0,-1.000000,0.000000,0.000000,8.000000
PAY_5,30000.0,-0.266200,1.133187,-2.0,-1.000000,0.000000,0.000000,8.000000
PAY_6,30000.0,-0.291100,1.149988,-2.0,-1.000000,0.000000,0.000000,8.000000
PAY_OVERDUE_COUNT,30000.0,0.834200,1.554303,0.0,0.000000,0.000000,1.000000,6.000000
WEIGHTED_PAYMENT_HISTORY,30000.0,-0.295907,2.398339,-4.9,-1.900000,0.000000,0.133333,16.050000
AVG_PAY_AMT,30000.0,5275.232094,10137.946323,0.0,1113.291667,2397.166667,5583.916667,627344.333333


KNN, wykorzystując funkcję odległości, nie rozróżnia poszczególnych cech. Dlatego też kolumny o większych wartościach mogą sztucznie zawyżać odległość. Na potrzeby KNN powinniśmy postarać się zeskalować zmienne, aby uniknąć tego zjawiska.

In [12]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

In [13]:
pipeline = Pipeline(steps=[
    ("scaler", StandardScaler()),
    ("classifier", GridSearchCV(KNeighborsClassifier(), 
                                param_grid={
                                    "n_neighbors": range(1, 10)
                                }, cv=5, scoring="f1", 
                                verbose=1, n_jobs=6, refit=True))
])

In [14]:
pipeline.fit(X_train.drop(columns="DEFAULT"), 
             X_train["DEFAULT"])

Fitting 5 folds for each of 9 candidates, totalling 45 fits


[Parallel(n_jobs=6)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=6)]: Done  45 out of  45 | elapsed:   52.3s finished


Pipeline(memory=None,
         steps=[('scaler',
                 StandardScaler(copy=True, with_mean=True, with_std=True)),
                ('classifier',
                 GridSearchCV(cv=5, error_score=nan,
                              estimator=KNeighborsClassifier(algorithm='auto',
                                                             leaf_size=30,
                                                             metric='minkowski',
                                                             metric_params=None,
                                                             n_jobs=None,
                                                             n_neighbors=5, p=2,
                                                             weights='uniform'),
                              iid='deprecated', n_jobs=6,
                              param_grid={'n_neighbors': range(1, 10)},
                              pre_dispatch='2*n_jobs', refit=True,
                              return_train

In [15]:
f1_score(X_test["DEFAULT"],
         pipeline.predict(X_test.drop(columns="DEFAULT")))

0.46248230297310056

Prawdopodobnie powinniśmy także rozważyć inne strategie skalowania zmiennych, tym bardziej że w oczywisty sposób mają one ogromny wpływ na wyniki.